### Homework 4

Import and clean the data.

In [28]:
import numpy as np
import pandas as pd
#Used to split into train and test set
from sklearn.model_selection import train_testing_split
from sklearn.preprocessing import normalize


#Assumes that the dataset is in the current working directory under the name 'SkyData.csv'
df = pd.read_csv('data/skyserver.csv')

#Delete some irrelevant columns
df.drop(['objid', 'rerun', 'specobjid', 'plate', 'mjd'], axis=1, inplace=True)

#Remove the Quasar class from the dataframe (makes the problem binary classification)
#Change class labels from string to 0 and 1
df = df[df['class']!='QSO']
df['class'] = pd.factorize(df['class'])[0]

#Create a dataframe without the class label
#Convert the dataframe to a numpy array
#Add an extra column of ones to the data so the linear model has an offset
Adf = df.drop('class', axis = 1)
A = Adf.to_numpy()
A = normalize(A)
A = np.hstack((A, np.ones((A.shape[0], 1), dtype=A.dtype)))

#Create a dataframe of only the class labels
#Convert ydf to a numpy array
ydf = df['class']
y = ydf.to_numpy()

#Split data into train and test set
A_train,A_test,y_train,y_test = train_testing_split(A, y, testing_size=0.2)
print(A_train.shape, A_test.shape, y_train.shape, y_test.shape)

ImportError: cannot import name 'train_testing_split' from 'sklearn.model_selection' (C:\Users\joela\anaconda3\envs\STOR712\lib\site-packages\sklearn\model_selection\__init__.py)

### Solutions

Implement the following:
1. Standard minibatch stochastic gradient
2. Mini-batch stochastic gradient via random reshuffling
3. SVRG

Begin by defining the L2-regularized logistic regression model.
Given data A and y and regularization parameter mu, compute either the function value or gradient for parameters x.

In [ ]:
def logistic_regression_model(x, mode, A, y, mu):
    """
    :param x: input value at which to evaluate function model
    :param A: data feature matrix
    :param y: data labels
    :param mu: L2 regularization parameter
    :return: logistic regression model value or gradient at parameters x
    """

    # compute predicted values for parameters x given data A
    predicted_values = np.matmul(A, x)

    # compute either the function value or gradient of the logistic regression model
    if mode == 'value':
        output = (
            np.mean(
                -1 * y * predicted_values + np.log(1 + np.exp(predicted_values))
            )
            + mu / 2 * np.linalg.norm(x)
        )
    elif mode == 'gradient':
        scalars = (-1 * y + (np.exp(predicted_values) / (1 + np.exp(predicted_values))))
        output = np.matmul(A.T, scalars) / A.shape[0] + mu * x
    else:
        raise ValueError('mode must be either "value" or "gradient"')

    return output


# each epoch consists of a full batch gradient update
def gradient_descent(
    fun,  # objective function
    A_train, A_test, y_train, y_test,  # data
    x0, alpha,  # algorithmic initializations
    total_epochs,
    decay=False  # learning rate decay of form alpha_k = alpha / k
):

    # initialize algorithm at initial point
    x = x0
    alpha_bar = alpha
    training_loss = fun(x, 'value', A_train, y_train)
    testing_loss = fun(x, 'value', A_test, y_test)

    # initialize objects to store output
    training_losses = [training_loss]
    testing_losses = [testing_loss]

    # run until maximum iterations are reached
    for epoch_number in range(total_epochs):

        # compute gradient for all observations
        gradient = fun(
            x, 'gradient',  # compute
            A_train, y_train  # data
        )

        # perform gradient step
        if decay:
            alpha = alpha_bar / (epoch_number + 1)
        x -= alpha * gradient

        # update training and test losses at batch end
        training_loss = fun(x, 'value', A_train, y_train)
        testing_loss = fun(x, 'value', A_test, y_test)

        # store in output
        training_losses.append(training_loss)
        testing_losses.append(testing_losses)


    return np.array(training_losses), np.array(testing_losses)

The following code chunks implement each of the new algorithmic implementations

In [ ]:
# each epoch consists of the number of batches that fit the input data
# each batch is randomly sampled without replacement
def stochastic_gradient_uniformly_at_random(
    fun,  # objective function
    A_train, A_test, y_train, y_test,  # data
    x0, alpha,  # algorithmic initializations
    total_epochs, batch_size,
    decay=False  # learning rate decay of form alpha_k = alpha / k
):

    # initialize algorithm at initial point
    x = x0
    alpha_bar = alpha
    training_loss = fun(x, 'value', A_train, y_train)
    testing_loss = fun(x, 'value', A_test, y_test)

    N = A_train.shape[0]
    total_batches = N // batch_size

    # initialize objects to store output
    training_losses = [training_loss]
    testing_losses = [testing_loss]

    # run until maximum iterations are reached
    for epoch_number in range(total_epochs):

        # update learning rate
        if decay:
            alpha = alpha_bar / (epoch_number + 1)

        for batch_number in range(total_batches):

            # select random minibatch of size batch_size
            batch_indices = np.random.permutation(N)[:batch_size]

            # compute gradient for randomly selected observations
            gradient = fun(
                x, 'gradient',  # compute
                A_train[batch_indices, :], y_train[batch_indices, :]  # data subset
            )

            # perform gradient step
            x -= alpha * gradient


        # update training and test losses at batch end
        training_loss = fun(x, 'value', A_train, y_train)
        testing_loss = fun(x, 'value', A_test, y_test)

        # store in output
        training_losses.append(training_loss)
        testing_losses.append(testing_losses)


    return np.array(training_losses), np.array(testing_losses)

In [ ]:
# each epoch consists of a random permutation of the data set
# batches of size batch_size are processed at a time moving through the permutation
# remaining observations smaller than the desired batch size will be discarded
def stochastic_gradient_random_reshuffle(
    fun,  # objective function
    A_train, A_test, y_train, y_test,  # data
    x0, alpha,  # algorithmic initializations
    total_epochs, batch_size,
    decay=False  # learning rate decay of form alpha_k = alpha / k
):

    # initialize algorithm at initial point
    x = x0
    alpha_bar = alpha
    training_loss = fun(x, 'value', A_train, y_train)
    testing_loss = fun(x, 'value', A_test, y_test)

    N = A_train.shape[0]
    total_batches = N // batch_size

    # initialize objects to store output
    training_losses = [training_loss]
    testing_losses = [testing_loss]

    # run until maximum iterations are reached
    for epoch_number in range(total_epochs):

        # randomly permute the data at the beginning of each epoch
        # note that data NOT divisible by the batch size will have data discarded
        epoch_permutation = np.random.permutation(N)
        A_train = A_train[epoch_permutation, :]
        y_train = y_train[epoch_permutation, :]

        # update learning rate
        if decay:
            alpha = alpha_bar / (epoch_number + 1)

        for batch_number in range(total_batches):

            # identify the chunk of the permutation to use for this batch
            batch_indices = np.arange(
                start = batch_number * batch_size,
                stop = (batch_number + 1) * batch_size
            )

            # compute gradient for randomly selected observations
            gradient = fun(
                x, 'gradient',  # compute
                A_train[batch_indices, :], y_train[batch_indices, :]  # data subset
            )

            # perform gradient step
            x -= alpha * gradient


        # update training and test losses at batch end
        training_loss = fun(x, 'value', A_train, y_train)
        testing_loss = fun(x, 'value', A_test, y_test)

        # store in output
        training_losses.append(training_loss)
        testing_losses.append(testing_losses)


    return np.array(training_losses), np.array(testing_losses)

In [ ]:
# source: Alg 5.1 of arxiv.org/pdf/1606.04838.pdf
def stochastic_variance_reduced_gradient(
    fun,  # objective function
    A_train, A_test, y_train, y_test,  # data
    x0, alpha,  # algorithmic initializations
    total_epochs, m,  # additional hyper-parameters
    decay=False  # learning rate decay of form alpha_k = alpha / k
):

    # initialize algorithm at initial point
    x_outer, x_inner = x0
    alpha_bar = alpha

    N = A_train.shape[0]

    # initialize objects to store output
    training_loss = fun(x_outer, 'value', A_train, y_train)
    testing_loss = fun(x_outer, 'value', A_test, y_test)

    training_losses = [training_loss]
    testing_losses = [testing_loss]

    # run until maximum iterations are reached
    for epoch_number in range(total_epochs):

        # compute the full batch gradient
        full_gradient = fun(
            x_outer, 'gradient',  # compute
            A_train, y_train  # data
        )

        # inner for loop iterations controlled by hyperparameter m
        for j in range(m):

            # identify the chunk of the permutation to use for this batch
            observation_index = np.random.choice(N)

            # compute gradient for randomly selected observation
            # with respect to both inner and outer iterations
            outer_gradient = fun(
                x_outer, 'gradient',  # compute
                A_train[observation_index, :], y_train[observation_index, :]  # data subset
            )

            inner_gradient = fun(
                x_inner, 'gradient',  # compute
                A_train[observation_index, :], y_train[observation_index, :]  # data subset
            )

            # compute gradient estimator
            variance_reduced_gradient = inner_gradient - (outer_gradient - full_gradient)

            # perform gradient step
            x_inner -= alpha * variance_reduced_gradient


        # update training and test losses at batch end
        x_outer = x_inner
        training_loss = fun(x_outer, 'value', A_train, y_train)
        testing_loss = fun(x_outer, 'value', A_test, y_test)

        # store in output
        training_losses.append(training_loss)
        testing_losses.append(testing_losses)


    return np.array(training_losses), np.array(testing_losses)

Perform the various descent methods for 100 total epochs with a batch size of 64 and a small regularization parameter of 10e-3.

In [ ]:
# defaults
total_epochs = 100
batch_size = 64
mu = 10e-3

In [ ]:
# tune-able hyper-pameters
x0 = np.zeros(A_train.shape[-1])
alpha = 0.01

# standard gradient descent
gradient_descent_training_losses, gradient_descent_testing_losses = gradient_descent(
    lambda x, y, A, b: logistic_regression_model(x, y, A, b, mu),
    A_train, A_test, y_train, y_test,
    x0, alpha,
    total_epochs
)

methods = ['Fixed GD']
methods_training_losses = [gradient_descent_training_losses]
methods_testing_losses = [gradient_descent_testing_losses]


Plot the various training and testing losses on seperate plots

In [ ]:
import matplotlib.pyplot as plt
fig, axarr = plt.subplots(ncols=2, sharey=True)

# plot training losses
plt.sca(axarr[0])
for method_training_losses in methods_training_losses:
    plt.plot(method_training_losses)

# plot testing losses
plt.sca(axarr[1])
for method_testing_losses in methods_testing_losses:
    plt.plot(method_testing_losses)

plt.legend(methods)
plt.show()
